In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
from pathlib import Path
import yaml
from subprocess import check_call

import inspect
import cesm_tools

## Check out model code and configure sandbox 

In [21]:
remote = 'git@github.com:ESCOMP/CAM.git'
coderoot = '/glade/p/cgd/oce/people/mclong/cesm'
cesmtag = 'cam6_3_027'

cesmroot = cesm_tools.code_checkout(remote, coderoot, cesmtag)

Check for right tag: /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027
HEAD detached at cam6_3_027
nothing to commit, working tree clean




## Set up local directory and machine-dependent paths

In [22]:
USER = os.environ["USER"]
scriptroot = os.getcwd()
os.makedirs(f"{scriptroot}/chem_mech_input", exist_ok=True)

machine = "cheyenne"
if machine in ["cheyenne"]:
    inputdata = "/glade/p/cesmdata/cseg/inputdata"
    scratch = f"/glade/scratch/{USER}"
    project_data = '/glade/work/mclong/cam-sd-cases/data' 
    caseroot_root = f'/glade/work/{USER}/cam-sd-cases'
else:
    raise ValueError(f"unknown machine: {machine}")

## Setup case options

In [23]:
clobber = True  # remove an existing case of this name

case = "cam6327_sd.apo_test_srf_emis.004"

compset = "SDYN_CAM60_CLM50%SP_CICE%PRES_DOCN%DOM_MOSART_SGLC_SWAV"
resolution = "f09_f09_mg17"
PROJECT = "NEOL0004"

chem_mech_template = f"{cesmroot}/src/chemistry/pp_trop_mam4/chem_mech.in"
assert os.path.exists(chem_mech_template)

# initial conditions
ncdata = f"{project_data}/f.e20.FWAMIP.f09_f09.134.1975.009.cam.i.2010-01-01_56L_c170403.apo-forward-tracers.nc"

# sathist output
if True:
    sathist_track_infile = f"{project_data}/sathist_infile.hip-orc-tom-aoa.20220303.nc"
else:
    sathist_track_infile = "none"

with open("ObsPack-lonlat-fincl-specifier.yml") as fid:
    fincl2lonlat_dict = yaml.safe_load(fid)
fincl2lonlat = ",".join([f"'{v}'" for v in fincl2lonlat_dict.values()])

# additional tracers
srf_emis_files = {}
if True:
    with open("srf-emis-apo-forward.yml", "r") as fid:
        srf_emis_files = yaml.safe_load(fid)

# the basic variables
sathist_fincl = ["T", "U", "V", "Q", "PS"]
fincl1 = ["U", "V", "T", "Q", "PS"]
fincl2 = ["U", "V", "T", "Q", "PS"]


# paths
caseroot = f"{caseroot_root}/cam-sd-cases/{case}"
lockroot = f"{caseroot_root}/cam-sd-cases/locked_cases"
rundir = f"{scratch}/{case}"
archive_root = f"{scratch}/archive/{case}"


# sst forcing
prescribed_sst = "custom"
sst_dataset = f"{inputdata}/atm/cam/sst/sst_HadOIBl_bc_0.9x1.25_1850_2020_c210521.nc"
sst_year_start = 1850
sst_year_end = 2020

# run settings
run_startdate = "2016-01-01"
resubmit = 0
stop_n = 1
stop_option = "nmonths"
rest_n = 12
rest_option = "nmonths"
timewall = "04:00:00"
queue = "economy"
PBS_ACCOUNT = PROJECT
PROJECT = PROJECT

os.makedirs("caseinfo", exist_ok=True)
with open(f"caseinfo/{case}.yml", "w") as fid:
    yaml.dump(
        dict(
            case=case,
            caseroot=caseroot,
            rundir=rundir,
            archive_root=archive_root,
            compset=compset,
            resolution=resolution,
            sathist_track_infile=sathist_track_infile,
            chem_mech_template=chem_mech_template,
            fincl2lonlat_dict=fincl2lonlat_dict,
            srf_emis_files=srf_emis_files,
            sst_dataset=sst_dataset,
            ncdata=ncdata,
        ),
        fid,
    )


## Create case

### Determine if existing case exists, remove if `clobber=True`.

In [24]:
# ensure root directories
os.makedirs(os.path.dirname(caseroot), exist_ok=True)
os.makedirs(lockroot, exist_ok=True)

lockfile = f"{lockroot}/{case}"
if clobber:
    check_call(['rm', '-f', lockfile])
    
create_build_case = not os.path.exists(lockfile)

if create_build_case:
    if os.path.exists(caseroot):
        print(f"deleting {caseroot}")        
        check_call(['rm', '-fr', caseroot])
    if os.path.exists(rundir):        
        print(f"deleting {rundir}")    
        check_call(['rm', '-fr', rundir])  

deleting /glade/work/mclong/cam-sd-cases/cam-sd-cases/cam6327_sd.apo_test_srf_emis.003
deleting /glade/scratch/mclong/cam6327_sd.apo_test_srf_emis.003


In [25]:
if create_build_case:
    check_call(
        [
            "./create_newcase",
            "--case",
            caseroot,
            "--project",
            PROJECT,
            "--machine",
            machine,
            "--res",
            resolution,
            "--compset",
            compset,
            "--run-unsupport",
        ],
        cwd=f"{cesmroot}/cime/scripts",
    )


Did not find an alias or longname compset match for SDYN_CAM60_CLM50%SP_CICE%PRES_DOCN%DOM_MOSART_SGLC_SWAV 
Automatically adding SIAC to compset
Automatically adding SESP to compset
Compset forcing is 


ATM component is CAM cam6 physics:
LND component is clm5.0:Satellite phenology:
ICE component is Sea ICE (cice) model version 5 :prescribed cice
OCN component is DOCN   prescribed ocean mode
ROF component is MOSART: MOdel for Scale Adaptive River Transport
GLC component is Stub glacier (land ice) component
WAV component is Stub wave component
IAC component is Stub iac component
ESP component is Stub external system processing (ESP) component
Pes     specification file is /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/cime_config/config_pes.xml
Compset specific settings: name is SSTICE_DATA_FILENAME and value is $DIN_LOC_ROOT/atm/cam/sst/sst_HadOIBl_bc_0.9x1.25_clim_c040926.nc
Compset specific settings: name is SSTICE_GRID_FILENAME and value is $DIN_LOC_ROOT/share/domains/domain.ocn.fv0.9x1.25_gx1v7.151020.nc
Machine is cheyenne
Pes setting: grid match    is a%0.9x1.25 
Pes setting: machine match is cheyenne 
Pes setting: grid          is a%0.9x1.25_l%0.9x1.25_oi%0.9x1.25_r%r05_g%null_w

## Configure case

### Generate a `chem_mech.in` file with additional tracers

In [26]:
srf_emis_files

{'apo_jena': {'constituent': 'O2',
  'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFAPO_OCN.carboscope.apo99X_v2021.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFAPO_OCN'},
 'co2_cesm': {'constituent': 'CO2',
  'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFCO2_OCN.cesm_fosi_smyle.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFCO2_OCN'},
 'co2_gridfed': {'constituent': 'CO2',
  'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFCO2_FF.GCP-GridFED.v2021.3.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFCO2_FF'},
 'co2_oco2mip': {'constituent': 'CO2',
  'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFCO2_FF.OCO2-MIP.v2020.1.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFCO2_FF'},
 'co2_somffn': {'constituent': 'CO2',
  'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFCO2_OCN.MPI-SOM-FFN.0.9x1.25.19860101-20201231.nc',
  'flux_var

In [27]:
chem_mech_file = f"{scriptroot}/chem_mech_input/chem_mech_{case}.in"

if create_build_case:
    chem_mech = cesm_tools.get_chem_mech_content(chem_mech_template)

    srf_emis_specifier_add = []
    if srf_emis_files:
        
        for var, info in srf_emis_files.items():
            path = info["flux_file"]
            constituent = info["constituent"]
            assert os.path.exists(path)
            srf_emis_specifier_add.append(f"'{var} -> {path}'")

            chem_mech["species"]["solution"].insert(0, f"{var} -> {constituent},")
            chem_mech["solution classes"]["explicit"].insert(0, f"{var}")
            
            fincl1.extend([var, f'SF{var}', f'TM{var}'])
            fincl2.append(var)
            sathist_fincl.append(var)
            
    cesm_tools.write_chem_mech(chem_mech, chem_mech_file)

os.environ["CHEM_MECH_IN"] = chem_mech_file

!cat ${CHEM_MECH_IN}

* Comments
* User-given Tag Description: CAM_MAM4_2021
* Tag database identifier : MZ290_CAM_MAM4_20210504
* Tag created by : lke
* Tag created from branch : CAM_MAM4
* Tag created on : 2021-05-03 18:08:30.812686-06
* Comments for this tag follow:
*     lke : 2021-05-03 : specified oxidants mechanism for CAM

SPECIES

  SOLUTION
    o2_gridfed -> O2,
    o2_gk01r16 -> O2,
    o2_cesm -> O2,
    n2_era -> N2,
    n2_cesm -> N2,
    co2_somffn -> CO2,
    co2_oco2mip -> CO2,
    co2_gridfed -> CO2,
    co2_cesm -> CO2,
    apo_jena -> O2,
    bc_a1 -> C,
    bc_a4 -> C,
    DMS -> CH3SCH3,
    dst_a1 -> AlSiO5,
    dst_a2 -> AlSiO5,
    dst_a3 -> AlSiO5,
    H2O2,
    H2SO4,
    ncl_a1 -> NaCl,
    ncl_a2 -> NaCl,
    ncl_a3 -> NaCl,
    num_a1 -> H,
    num_a2 -> H,
    num_a3 -> H,
    num_a4 -> H,
    pom_a1 -> C,
    pom_a4 -> C,
    SO2,
    so4_a1 -> NH4HSO4,
    so4_a2 -> NH4HSO4,
    so4_a3 -> NH4HSO4,
    SOAG -> C,
    soa_a1 -> C,
    soa_a2 -> C,
    H2O
  END SOLUTION

  FIX

### Apply XML settings

In [28]:
xmlchange = lambda arg: check_call(["./xmlchange", arg], cwd=caseroot)

if create_build_case:
    xmlchange("NTASKS_ATM=720,NTASKS_LND=720,NTASKS_ICE=720,NTASKS_OCN=720")
    xmlchange("NTASKS_CPL=720,NTASKS_GLC=1,NTASKS_ROF=720,NTASKS_WAV=720")

    xmlchange("PIO_TYPENAME=pnetcdf")

    # Configure option (env_conf.xml)
    xmlchange(f"RUN_STARTDATE={run_startdate}")

    # Change CAM Configure here.
    xmlchange(
        f"CAM_CONFIG_OPTS=-nlev 56 -offline_dyn -usr_mech_infile {chem_mech_file}"
    )
    xmlchange("CALENDAR=GREGORIAN")

    # Switch SST file to evolving
    xmlchange(f"SSTICE_DATA_FILENAME={sst_dataset}")
    xmlchange(f"SSTICE_YEAR_END={sst_year_end}")
    xmlchange(f"SSTICE_YEAR_START={sst_year_start}")
    xmlchange(f"SSTICE_YEAR_ALIGN={sst_year_start}")
else:
    xmlchange("CONTINUE_RUN=TRUE")


xmlchange(f"RESUBMIT={resubmit}")
xmlchange(f"STOP_OPTION={stop_option}")
xmlchange(f"STOP_N={stop_n}")
xmlchange(f"REST_OPTION={rest_option}")
xmlchange(f"REST_N={rest_n}")
xmlchange(f"JOB_WALLCLOCK_TIME={timewall}")
xmlchange(f"JOB_QUEUE={queue}");

## Run setup

In [29]:
if create_build_case:
    check_call(['./case.setup'], cwd=caseroot)

Setting Environment OMP_STACKSIZE=1024M
Setting Environment TMPDIR=/glade/scratch/mclong
Setting Environment MPI_TYPE_DEPTH=16
Setting Environment MPI_IB_CONGESTED=1
Setting resource.RLIMIT_STACK to -1 from (307200000, -1)


/glade/work/mclong/cam-sd-cases/cam-sd-cases/cam6327_sd.apo_test_srf_emis.003/env_mach_specific.xml already exists, delete to replace
/glade/work/mclong/cam-sd-cases/cam-sd-cases/cam6327_sd.apo_test_srf_emis.003/env_mach_specific.xml already exists, delete to replace


job is case.run USER_REQUESTED_WALLTIME 04:00:00 USER_REQUESTED_QUEUE premium WALLTIME_FORMAT %H:%M:%S
Creating batch scripts
Writing case.run script from input template /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/cime/config/cesm/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/cime/config/cesm/machines/template.st_archive
Creating file case.st_archive
Creating user_nl_xxx files for components and cpl
If an old case build already exists, might want to run 'case.build --clean' before building
You can now run './preview_run' to get more info on how your case will be run


## Namelist settings

### External forcing tracers

In [30]:
ext_frc_specifier = [
    f"'H2O -> {inputdata}/atm/cam/chem/emis/elev/H2OemissionCH4oxidationx2_3D_L70_1849-2101_CMIP6ensAvg_SSP2-4.5_c190403.nc'",
    f"'num_a1 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_so4_a1_anthro-ene_vertical_mol_175001-210101_0.9x1.25_c20201129.nc'",
    f"'num_a1 -> {inputdata}/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_num_so4_a1_contvolcano_vertical_1750-2100_0.9x1.25_c20170322.nc'",
    f"'num_a2 -> {inputdata}/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_num_so4_a2_contvolcano_vertical_1750-2100_0.9x1.25_c20170322.nc'",
    f"'SO2 -> {inputdata}/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_SO2_contvolcano_vertical_1750-2100_0.9x1.25_c20170322.nc'",
    f"'so4_a1 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_so4_a1_anthro-ene_vertical_mol_175001-210101_0.9x1.25_c20201129.nc'",
    f"'so4_a1 -> {inputdata}/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_so4_a1_contvolcano_vertical_1750-2100_0.9x1.25_c20170322.nc'",
    f"'so4_a2 -> {inputdata}/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_so4_a2_contvolcano_vertical_1750-2100_0.9x1.25_c20170322.nc'",
]

### Surface Emissions Tracers

In [31]:
srf_emis_specifier_default = [
    f"'bc_a4 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_bc_a4_anthro_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'bc_a4 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_bc_a4_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'DMS -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_DMS_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'DMS -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-SSP_DMS_other_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'num_a1 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_so4_a1_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'num_a1 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_so4_a1_anthro-ag-ship_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'num_a2 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_so4_a2_anthro-res_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'num_a4 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_bc_a4_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'num_a4 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_bc_a4_anthro_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'num_a4 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_pom_a4_anthro_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'num_a4 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_pom_a4_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'pom_a4 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_pom_a4_anthro_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'pom_a4 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_pom_a4_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'SO2 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_SO2_anthro-ag-ship-res_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'SO2 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_SO2_anthro-ene_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'SO2 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_SO2_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'so4_a1 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_so4_a1_anthro-ag-ship_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'so4_a1 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_so4_a1_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'so4_a2 -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_so4_a2_anthro-res_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'SOAG -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_SOAGx1.5_anthro_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'SOAG -> {inputdata}/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_SOAGx1.5_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
    f"'SOAG -> {inputdata}/atm/cam/chem/emis/emissions_ssp/emissions-cmip6-SOAGx1.5_biogenic_surface_mol_175001-210101_0.9x1.25_c20190329.nc'",
]

srf_emis_specifier = srf_emis_specifier_default + srf_emis_specifier_add
srf_emis_specifier    

["'bc_a4 -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_bc_a4_anthro_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
 "'bc_a4 -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_bc_a4_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
 "'DMS -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_DMS_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
 "'DMS -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-SSP_DMS_other_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
 "'num_a1 -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_so4_a1_bb_surface_mol_175001-210101_0.9x1.25_c20190222.nc'",
 "'num_a1 -> /glade/p/cesmdata/cseg

### Fixed Lower Boundary Condition Tracers

In [32]:
flbc_file_default = f'{inputdata}/atm/waccm/lb/LBC_1750-2500_CMIP6_SSP245_0p5degLat_GlobAnnAvg_c190828.nc'
flbc_list_default = ['CO2','CH4','N2O','CFC11eq','CFC12']

# make modifications

flbc_file = flbc_file_default
flbc_list = list(flbc_list_default)

### Write namelist modifcations

In [33]:
user_nl = dict()

if sathist_track_infile != 'none':
    sathist_fincl_str = ','.join([f"'{v}'" for v in sathist_fincl])
else:
    sathist_fincl_str = "'none'"
    
user_nl['cam'] = f"""
&camexp
  ncdata = '{ncdata}'
  bnd_topo = '/glade/work/tilmes/inputdata/topo/fv_0.9x1.25_nc3000_Nsw042_Nrs008_Co060_Fi001_ZR_merra2.nc'
  dust_emis_fact = 0.7D0
  soil_erod_file = '/glade/u/home/mahowald/dst_source2x2tunedcam6-2x2-04062017.nc'
  ext_frc_type = 'SERIAL'
  ext_frc_specifier = {', '.join(ext_frc_specifier)}
  srf_emis_type = 'INTERP_MISSING_MONTHS'
  srf_emis_specifier = {', '.join(srf_emis_specifier)}
  flbc_file = '{flbc_file}'
  flbc_list = {', '.join([f"'{v}'" for v in flbc_list])}
  flbc_type = 'SERIAL'
  scenario_ghg = 'CHEM_LBC_FILE'
/
&metdata_nl
  met_nudge_temp = .false.
  met_data_file = '{run_startdate[:4]}/MERRA2_0.9x1.25_{run_startdate[:4]}0101.nc'
  met_data_path = '{inputdata}/atm/cam/met/MERRA2/0.9x1.25/'
  met_filenames_list = '{inputdata}/atm/cam/met/MERRA2/0.9x1.25/filenames_1975-2021_c220109.txt'
  met_fix_mass = .true.
  met_qflx_factor = 1.0
  met_rlx_time = 24.
  met_srf_land = .false.
  met_srf_land_scale = .false.
  met_srf_nudge_flux = .false.
  met_srf_rad = .false.
  met_srf_refs = .false.
  met_srf_sst = .false.
  met_srf_tau = .false.
/
&rad_cnst_nl
 rad_diag_1 = 'A:Q:H2O', 'N:O2:O2', 'N:CO2:CO2', 'N:ozone:O3', 'N:N2O:N2O', 'N:CH4:CH4', 
              'N:CFC11:CFC11', 'N:CFC12:CFC12'
/ 
&satellite_options_nl
  sathist_mfilt = 10000
  sathist_track_infile = '{sathist_track_infile}'
  sathist_fincl = {sathist_fincl_str}
/
&cam_history_nl
  fincl1 = {','.join([f"'{v}'" for v in fincl1])}
  fincl2 = {','.join([f"'{v}'" for v in fincl2])}
  fincl2lonlat = {fincl2lonlat}
  nhtfrq = -6,-1
  mfilt  = 4,4
/
"""

user_nl['clm'] = f"""
&clm_inparm
  use_init_interp = .true.
/
"""

if create_build_case:
    for key, nl in user_nl.items():
        with open(f"{caseroot}/user_nl_{key}", 'w') as fid:
            fid.write(user_nl[key])
            
os.environ['CAT_FILE'] = f"{caseroot}/user_nl_cam"
!cat ${CAT_FILE}


&camexp
  ncdata = '/glade/work/mclong/cam-sd-cases/data/f.e20.FWAMIP.f09_f09.134.1975.009.cam.i.2010-01-01_56L_c170403.apo-forward-tracers.nc'
  bnd_topo = '/glade/work/tilmes/inputdata/topo/fv_0.9x1.25_nc3000_Nsw042_Nrs008_Co060_Fi001_ZR_merra2.nc'
  dust_emis_fact = 0.7D0
  soil_erod_file = '/glade/u/home/mahowald/dst_source2x2tunedcam6-2x2-04062017.nc'
  ext_frc_type = 'SERIAL'
  ext_frc_specifier = 'H2O -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/elev/H2OemissionCH4oxidationx2_3D_L70_1849-2101_CMIP6ensAvg_SSP2-4.5_c190403.nc', 'num_a1 -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/emissions_ssp245/emissions-cmip6-ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_num_so4_a1_anthro-ene_vertical_mol_175001-210101_0.9x1.25_c20201129.nc', 'num_a1 -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_num_so4_a1_contvolcano_vertical_1750-2100_0.9x1.25_c20170322.nc', 'num_a2 -> /glade/p/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_e

## Build the model

In [34]:
if create_build_case:
    with open(f"{caseroot}/Macros.make", 'a') as fid:
        fid.write("HAS_F2008_CONTIGUOUS := FALSE\n")
    check_call(['qcmd', '--', './case.build'], cwd=caseroot)
    Path(f"{lockroot}/{case}.locked").touch()

Submitting command to PBS using account NEOL0001:
    ./case.build

Waiting for job 3142191.chadmin1.ib0.cheyenne.ucar.edu to start ... 


Building case in directory /glade/work/mclong/cam-sd-cases/cam-sd-cases/cam6327_sd.apo_test_srf_emis.003
sharedlib_only is False
model_only is False
Setting Environment OMP_STACKSIZE=1024M
Setting Environment TMPDIR=/glade/scratch/mclong
Setting Environment MPI_TYPE_DEPTH=16
Setting Environment MPI_IB_CONGESTED=1
Setting resource.RLIMIT_STACK to -1 from (-1, -1)
Generating component namelists as part of build
  2022-03-04 15:02:42 atm 
Create namelist for component cam
   Calling /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/cime_config/buildnml
     ...calling cam buildcpp to set build time options
CAM namelist copy: file1 /glade/work/mclong/cam-sd-cases/cam-sd-cases/cam6327_sd.apo_test_srf_emis.003/Buildconf/camconf/atm_in file2 /glade/scratch/mclong/cam6327_sd.apo_test_srf_emis.003/run/atm_in 
  2022-03-04 15:02:47 lnd 
Create namelist for compo

## Preview namelists

In [35]:
check_call(['./preview_namelists'], cwd=caseroot)

Setting Environment OMP_STACKSIZE=1024M
Setting Environment TMPDIR=/glade/scratch/mclong
Setting Environment MPI_TYPE_DEPTH=16
Setting Environment MPI_IB_CONGESTED=1
Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
  2022-03-04 15:13:25 atm 
Create namelist for component cam
   Calling /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/cime_config/buildnml
CAM namelist copy: file1 /glade/work/mclong/cam-sd-cases/cam-sd-cases/cam6327_sd.apo_test_srf_emis.003/Buildconf/camconf/atm_in file2 /glade/scratch/mclong/cam6327_sd.apo_test_srf_emis.003/run/atm_in 
  2022-03-04 15:13:26 lnd 
Create namelist for component clm
   Calling /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/components/clm//cime_config/buildnml
  2022-03-04 15:13:26 ice 
Create namelist for component cice
   Calling /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/components/cice//cime_config/buildnml
  2022-03-04 15:13:27 ocn 
Create namelist for component docn
   Calling /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/ci

   NOTE: ignoring setting of rof2ocn_fmapname=idmap in seq_maps.rc
   NOTE: ignoring setting of rof2ocn_ice_rmapname=idmap in seq_maps.rc
   NOTE: ignoring setting of rof2ocn_liq_rmapname=idmap in seq_maps.rc


0

## Submit the job

In [36]:
check_call(['./case.submit'], cwd=caseroot)


env_batch.xml appears to have changed, regenerating batch scripts
manual edits to these file will be lost!



Creating batch scripts
Writing case.run script from input template /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/cime/config/cesm/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/cime/config/cesm/machines/template.st_archive
Creating file case.st_archive
Setting Environment OMP_STACKSIZE=1024M
Setting Environment TMPDIR=/glade/scratch/mclong
Setting Environment MPI_TYPE_DEPTH=16
Setting Environment MPI_IB_CONGESTED=1
Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
  2022-03-04 15:13:29 atm 
Create namelist for component cam
   Calling /glade/p/cgd/oce/people/mclong/cesm/cam6_3_027/cime_config/buildnml
CAM namelist copy: file1 /glade/work/mclong/cam-sd-cases/cam-sd-cases/cam6327_sd.apo_test_srf_emis.003/Buildconf/camconf/atm_in file2 /glade/scratch/mclong/cam6327_sd.apo_test_srf_emis.003/run/atm_in 
  2022-03-04 15:13:29 lnd 
Create namelist for component clm
   Calling /glade/p

   NOTE: ignoring setting of rof2ocn_fmapname=idmap in seq_maps.rc
   NOTE: ignoring setting of rof2ocn_ice_rmapname=idmap in seq_maps.rc
   NOTE: ignoring setting of rof2ocn_liq_rmapname=idmap in seq_maps.rc


Checking that inputdata is available as part of case submission
Setting Environment OMP_STACKSIZE=1024M
Setting Environment TMPDIR=/glade/scratch/mclong
Setting Environment MPI_TYPE_DEPTH=16
Setting Environment MPI_IB_CONGESTED=1
Setting resource.RLIMIT_STACK to -1 from (-1, -1)
Loading input file list: 'Buildconf/ctsm.input_data_list'
Loading input file list: 'Buildconf/cpl.input_data_list'
Loading input file list: 'Buildconf/cice.input_data_list'
Loading input file list: 'Buildconf/docn.input_data_list'
Loading input file list: 'Buildconf/mosart.input_data_list'
Loading input file list: 'Buildconf/cam.input_data_list'
Check case OK
Submitting job script qsub -q premium -l walltime=04:00:00 -A NEOL0004 -m ea -v ARGS_FOR_SCRIPT='--resubmit' .case.run


submit_jobs case.run
Submit job case.run
Submit job case.st_archive


Submitted job id is 3142296.chadmin1.ib0.cheyenne.ucar.edu
Submitting job script qsub -q premium -l walltime=04:00:00 -A NEOL0004  -W depend=afterok:3142296.chadmin1.ib0.cheyenne.ucar.edu -m ea -v ARGS_FOR_SCRIPT='--resubmit' case.st_archive
Submitted job id is 3142297.chadmin1.ib0.cheyenne.ucar.edu
Submitted job case.run with id 3142296.chadmin1.ib0.cheyenne.ucar.edu
Submitted job case.st_archive with id 3142297.chadmin1.ib0.cheyenne.ucar.edu


0